In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split


In [11]:
dataset_path = r'C:\Users\Dell\Desktop\MANI\Null class tasks\3. Age and Emotion Detection for Movie Theatre\UTKFace'


In [12]:
def load_images_and_ages(dataset_path):
    images = []
    ages = []

    for filename in os.listdir(dataset_path):
        if filename.endswith(".jpg"):
            parts = filename.split('_')

            if len(parts) < 4:
                continue  # Skip files that don't follow the expected naming pattern

            try:
                age = int(parts[0])  # Extract age from the filename (first part)
                img_path = os.path.join(dataset_path, filename)
                img = cv2.imread(img_path)
                img = cv2.resize(img, (48, 48))  # Resize to 48x48
                img = img.astype('float32') / 255.0  # Normalize the image

                images.append(img)
                ages.append(age)
            except ValueError:
                continue  # Skip files with invalid age labels

    images = np.array(images)
    ages = np.array(ages)
    return images, ages


In [13]:
images, ages = load_images_and_ages(dataset_path)

In [14]:
max_age = ages.max()
ages_normalized = ages / max_age


In [15]:
X_train, X_val, y_train, y_val = train_test_split(images, ages_normalized, test_size=0.2, random_state=42)


In [16]:
def build_age_prediction_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 3)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1, activation='linear'))  # Single output neuron for regression
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
    return model

In [17]:
age_model = build_age_prediction_model()


C:\Users\Dell\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [18]:
age_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))


Epoch 1/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 23s 34ms/step - loss: 0.0346 - mae: 0.1371 - val_loss: 0.0133 - val_mae: 0.0853
Epoch 2/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 16s 27ms/step - loss: 0.0130 - mae: 0.0855 - val_loss: 0.0125 - val_mae: 0.0808
Epoch 3/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 16s 27ms/step - loss: 0.0101 - mae: 0.0748 - val_loss: 0.0089 - val_mae: 0.0684
Epoch 4/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 17s 28ms/step - loss: 0.0081 - mae: 0.0670 - val_loss: 0.0081 - val_mae: 0.0654
Epoch 5/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 16s 27ms/step - loss: 0.0071 - mae: 0.0625 - val_loss: 0.0084 - val_mae: 0.0654
Epoch 6/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 16s 27ms/step - loss: 0.0066 - mae: 0.0604 - val_loss: 0.0084 - val_mae: 0.0683
Epoch 7/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 17s 29ms/step - loss: 0.0062 - mae: 0.0588 - val_loss: 0.0074 - val_mae: 0.0645
Epoch 8/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 16s 28ms/step - loss: 0.0056 - mae: 0.0558 - val_loss: 0.0073 - val_mae: 0.0626
Epoch 9/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 

In [19]:
loss, mae = age_model.evaluate(X_val, y_val)
print(f"Validation Mean Absolute Error: {mae * max_age:.2f} years")


149/149 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 0.0075 - mae: 0.0624
Validation Mean Absolute Error: 7.25 years


In [20]:
age_model.save('age_detection_model.h5')
